#### Import packages

In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder # for encoding categorical labels
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer

#### Define the dataset

In [52]:
data = pd.read_csv('sentimentdataset.csv')
df = pd.DataFrame(data)
df.head()

,Unnamed: 0.1,Unnamed: 0,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour
0,0,0,Enjoying a beautiful day at the park! ...,Positive,2023-01-15 12:30:00,User123,Twitter,#Nature #Park,15.0,30.0,USA,2023,1,15,12
1,1,1,Traffic was terrible this morning. ...,Negative,2023-01-15 08:45:00,CommuterX,Twitter,#Traffic #Morning,5.0,10.0,Canada,2023,1,15,8
2,2,2,Just finished an amazing workout! 💪 ...,Positive,2023-01-15 15:45:00,FitnessFan,Instagram,#Fitness #Workout,20.0,40.0,USA,2023,1,15,15
3,3,3,Excited about the upcoming weekend getaway! ...,Positive,2023-01-15 18:20:00,AdventureX,Facebook,#Travel #Adventure,8.0,15.0,UK,2023,1,15,18
4,4,4,Trying out a new recipe for dinner tonight. ...,Neutral,2023-01-15 19:55:00,ChefCook,Instagram,#Cooking #Food,12.0,25.0,Australia,2023,1,15,19


In [53]:
print(df.info()) #check for null values and data types

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0.1  732 non-null    int64  
 1   Unnamed: 0    732 non-null    int64  
 2   Text          732 non-null    object 
 3   Sentiment     732 non-null    object 
 4   Timestamp     732 non-null    object 
 5   User          732 non-null    object 
 6   Platform      732 non-null    object 
 7   Hashtags      732 non-null    object 
 8   Retweets      732 non-null    float64
 9   Likes         732 non-null    float64
 10  Country       732 non-null    object 
 11  Year          732 non-null    int64  
 12  Month         732 non-null    int64  
 13  Day           732 non-null    int64  
 14  Hour          732 non-null    int64  
dtypes: float64(2), int64(6), object(7)
memory usage: 85.9+ KB
None


#### Data Processing

In [ ]:
labelenc = LabelEncoder() #label encoder instance
df['Sentiment_encoded'] = labelenc.fit_transform(df['Sentiment'])

df['combined'] = df['Text'].astype(str) + " " + df['Hashtags'].astype(str)


vecto = TfidfVectorizer()
text_vec = vecto.fit_transform(df['combined'])



#create a mapping of original labels to encoded values
mapping = {label: int(idx) for idx, label in enumerate(labelenc.classes_)}
print("Label -> Encoded mapping:")
for label, idx in mapping.items():
    print(f"{label} -> {idx}")


                                                Text  \
0   Enjoying a beautiful day at the park!        ...   
1   Traffic was terrible this morning.           ...   
2   Just finished an amazing workout! 💪          ...   
3   Excited about the upcoming weekend getaway!  ...   
4   Trying out a new recipe for dinner tonight.  ...   

                                     Hashtags  \
0   #Nature #Park                               
1   #Traffic #Morning                           
2   #Fitness #Workout                           
3   #Travel #Adventure                          
4   #Cooking #Food                              

                                            combined  
0   Enjoying a beautiful day at the park!        ...  
1   Traffic was terrible this morning.           ...  
2   Just finished an amazing workout! 💪          ...  
3   Excited about the upcoming weekend getaway!  ...  
4   Trying out a new recipe for dinner tonight.  ...  
Label -> Encoded mapping:
 Acceptance 